In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from tensorflow.python.keras.models import load_model
import tensorflow as tf

from google.colab import files

import pickle

In [3]:
path=f"/gdrive/My Drive/Colab Notebooks/dacon/credit_card"
test_path=path+"/test.csv"
csv_test=pd.read_csv(test_path)

In [4]:
csv_test

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
0,26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0
1,26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0
2,26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0
3,26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0
4,26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36452,F,Y,Y,0,202500.0,Working,Incomplete higher,Married,House / apartment,-18593,-5434,1,1,1,0,Accountants,2.0,-19.0
9996,36453,M,Y,Y,0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886,-1315,1,1,0,0,Laborers,2.0,-34.0
9997,36454,F,N,Y,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,-21016,-14018,1,0,0,0,Medicine staff,2.0,-55.0
9998,36455,F,Y,N,0,180000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-16541,-1085,1,0,1,0,NaN,2.0,-33.0


In [17]:
class Preprocess_class:
    def __init__(self,path):
        print("---------전처리를 시작합니다----------")
        self.my_csv=pd.read_csv(path)
        self.size=len(self.my_csv) # 데이터개수
        self.columns=self.my_csv.columns # 모든칼럼

        # 명목척도와 그외로 나누기 
        self.nominal=["gender","car","reality","income_type","edu_type","family_type","house_type","work_phone","phone","email","occyp_type"] # FLAG_MOBIL은 모두 1이므로 제외
        self.notNominal=["income_total","child_num","DAYS_BIRTH","DAYS_EMPLOYED","family_size","begin_month"]

        self.df=pd.DataFrame()
        self.df_y=pd.DataFrame()
        
        self.x=[]
        self.y=[]

    def getX(self):
        return self.x

    def getY(self):
        return self.y

    #------------------ x, y preprocesses------------------
    def pick_preprocessing_x(self,preprocessNum):
        if preprocessNum==1:
            self.preprocess_1_onehot()
        elif preprocessNum==2:
            self.preprocess_2_delCol()
        elif preprocessNum==3:
            self.preprocess_3_factorizing()
        elif preprocessNum==4:
            self.preprocess_1_onehot()
        elif preprocessNum==5:
            self.preprocess_4_bitmask()
        
    def pick_preproceesing_y(self,preprocessNum):
        if preprocessNum==1:
            self.preprocessing_y_dummy()
        elif preprocessNum==2:
            self.preprocessing_y_dummy()
        elif preprocessNum==3:
            self.preprocessing_y_dummy()
        elif preprocessNum==4:
            self.preprocessing_y_numbering()
        elif preprocessNum==5:
            self.preprocessing_y_numbering()
            
    #----------------types of preprocesses_x------------------
    def preprocess_1_onehot(self):
        self.nominal_onehot() # 명목척도 전처리
        self.notNominal_scaling() # 비명목척도 전처리

        self.transformation(self.df,self.x)
        self.x=np.array(self.x)
        
    def preprocess_2_delCol(self):
        delCol=["work_phone","phone","email"]
        for col in delCol:
            if col in self.nominal:
                self.nominal.remove(col)
            else:
                self.notNominal.remove(col)

        self.nominal_onehot() # 명목척도 전처리
        self.notNominal_scaling() # 비명목척도 전처리

        self.transformation(self.df,self.x)
        self.x=np.array(self.x)

    def preprocess_3_factorizing(self):
        self.nominal_factorizing()
        self.notNominal_scaling()
        
        self.transformation(self.df,self.x)
        self.x=np.array(self.x)

    def preprocess_4_bitmask(self):
        self.nominal_factorizing()
        self.bitmasking()

        self.notNominal_scaling()
        
        display(self.df)
        
        self.transformation(self.df,self.x)
        self.x=np.array(self.x)


    #----------------types of preprocesses_y------------------
    def preprocessing_y_dummy(self):
        self.df_y=pd.get_dummies(self.my_csv["credit"])
        self.df_y.columns=[0,1,2]
        self.transformation(self.df_y,self.y)
        self.y=np.array(self.y)

    def preprocessing_y_numbering(self):
        self.y=list(self.my_csv["credit"])
        self.y=np.array(list(map(str,self.y)))

        # L=[]
        # for i in range(self.size):
        #     L.append([self.my_csv["credit"][i]])
        # # self.y=np.array([self.my_csv["credit"]])
        # self.y=np.array(L)

    #--------------funcs------------------
    def bitmasking(self):
        uniqD={}
        for col in self.nominal:
            uniq=len(self.df[col].unique())
            print(uniq,col)
            if uniq in uniqD:
                uniqD[uniq].append(col)
            else:
                uniqD[uniq]=[col]
        print(uniqD)
        
        newDataFrame=pd.DataFrame()
        for domainNum in uniqD:
            tempD={domainNum:[]}
            for i in range(self.size):
                a=0
                for j,col in enumerate(uniqD[domainNum]):
                    a+=self.df[col][i]*(domainNum**(j))
                tempD[domainNum].append(a)

            tempD=pd.DataFrame(tempD)
            newDataFrame=pd.concat([newDataFrame,tempD],axis=1)

        self.df=newDataFrame
            


    def nominal_factorizing(self):
        for col in self.nominal:
            colDataFrame=pd.DataFrame(self.my_csv[col].factorize()[0],columns=[col])
            self.df=pd.concat([self.df,colDataFrame],axis=1)
            

    def nominal_numbering(self):
        for col in self.nominal:
            uniques=self.my_csv[col].unique()
            uniqueSize=len(uniques)
            d={}

            for i in range(uniqueSize):
                d[uniques[i]]=i
    
            for i in range(self.size):
                self.my_csv[col][i]=int(d[self.my_csv[col][i]])

            self.df=pd.concat([self.df,self.my_csv[col]],axis=1)
            self.df=self.df.astype({col:"int"})

    def nominal_onehot(self):
        # make nominal onehot
        for col in self.nominal:
            addCol=pd.get_dummies(self.my_csv[col])
            self.df=pd.concat([self.df,addCol],axis=1)

    def notNominal_scaling(self):
        # 양수로 바꾸기
        for i in range(self.size):
            self.my_csv["DAYS_BIRTH"][i]=abs(self.my_csv["DAYS_BIRTH"][i])

        for i in range(self.size):
            if self.my_csv["DAYS_EMPLOYED"][i]>0: 
                self.my_csv["DAYS_EMPLOYED"][i]=0
            else:
                self.my_csv["DAYS_EMPLOYED"][i]=abs(self.my_csv["DAYS_EMPLOYED"][i])
                
        # 스케일링
        for col in self.notNominal:
            addCol=pd.DataFrame(self.my_scaling(self.my_csv[col]))
            addCol.columns=[col]
            self.df=pd.concat([self.df,addCol],axis=1)

    # df -> np array
    def transformation(self,df,L):
        for i in range(self.size):
            temp_x=np.array(df.iloc[i])
            L.append(temp_x)

    # 스케일링
    def my_scaling(self,scaling_col):
        scaling_col=np.array(scaling_col).reshape(-1,1)
        
        scaler = preprocessing.MinMaxScaler()
        scaler_fit = scaler.fit(scaling_col)
        result=scaler_fit.transform(scaling_col)

        return result


# 1. 전처리방법 선택

# 2. 모델 선택

In [18]:
print('''
6 : dnn
7 : decision tree
8 : randomforest
9 : lightGBM
10 : XGBoost
11 : bitmast,xgboost
12 : logisticLinear
13 : gradient boosting
''')

finalModel=int(input("최종모델 번호를 입력히시오 : "))

if finalModel==6:
    preprocessNum,modelNum=3,2
elif finalModel==7:
    preprocessNum,modelNum=4,3
elif finalModel==8:
    preprocessNum,modelNum=4,4
elif finalModel==9:
    preprocessNum,modelNum=4,5
elif finalModel==10:
    preprocessNum,modelNum=4,6
elif finalModel==11:
    preprocessNum,modelNum=5,6
elif finalModel==12:
    preprocessNum,modelNum=4,7
elif finalModel==13:
    preprocessNum,modelNum=4,8

# 전처리
test_preprocess_object=Preprocess_class(path+"/test.csv")
test_preprocess_object.pick_preprocessing_x(preprocessNum)
x_test=test_preprocess_object.getX()

# 모델선택
with open(path+f"/models/model_{finalModel}.pickle", 'rb') as fr:
    model = pickle.load(fr)


6 : dnn
7 : decision tree
8 : randomforest
9 : lightGBM
10 : XGBoost
11 : bitmast,xgboost
12 : logisticLinear
13 : gradient boosting

최종모델 번호를 입력히시오 : 13
---------전처리를 시작합니다----------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# 3. 제출방법 선택
# (1.확률, 2.원핫)

In [19]:
a={"index":[i for i in range(26457,36457)]}
indexCol=pd.DataFrame(a)
size=36457-26457

algo=int(input("1:확률값으로, 2:onehotCoding"))

if algo==1:
    y_prob= model.predict_proba(x_test)
    y_prob=np.array(y_prob)
    print(y_prob.shape)
    print(y_prob)
    y_test=pd.DataFrame(columns=[str(i) for i in range(3)])
    for i in range(size):
        y_test=y_test.append( {str(j):y_prob[i][j] for j in range(3)}, ignore_index="True")
    print(y_test)
else:
    # y_pred_class=model.predict_classes(x_test) #dnn모델일때
    y_pred_class=model.predict(x_test) #tree일때
    print(y_pred_class)
    y_test=pd.get_dummies(y_pred_class)
    y_test.columns=[0,1,2]
    print(y_test)

1:확률값으로, 2:onehotCoding1
(10000, 3)
[[0.10208619 0.16593545 0.73197835]
 [0.103433   0.1693996  0.7271674 ]
 [0.10961137 0.18880957 0.70157906]
 ...
 [0.07085832 0.16127433 0.76786734]
 [0.08613921 0.16695255 0.74690824]
 [0.15783786 0.1538727  0.68828945]]
             0         1         2
0     0.102086  0.165935  0.731978
1     0.103433  0.169400  0.727167
2     0.109611  0.188810  0.701579
3     0.107733  0.151909  0.740358
4     0.121177  0.197062  0.681761
...        ...       ...       ...
9995  0.124049  0.200505  0.675446
9996  0.106298  0.241515  0.652187
9997  0.070858  0.161274  0.767867
9998  0.086139  0.166953  0.746908
9999  0.157838  0.153873  0.688289

[10000 rows x 3 columns]


# 4. 제출번호 선택

In [20]:
subNum=int(input("제출번호 : "))
sub=pd.concat([indexCol,y_test],axis=1)
sub.to_csv(path+f"/subs/sub_{subNum}.csv",index=False)

files.download(path+f"/subs/sub_{subNum}.csv")

제출번호 : 13


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>